In [10]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
from math import ceil

In [11]:
# CONFIG / DATA DIRECTORIES
# =========================
train_dir = '../data/train'
test_dir  = '../data/test'
val_dir   = '../data/val'
BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [12]:
# DATA AUGMENTATION (for classification)
# =========================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
# DATA LOADERS (classification generators with labels)
# =========================
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', shuffle=False
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [14]:
# CLASS WEIGHTS (handle imbalance)
# =========================
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))
print("Computed Class Weights:", class_weights)

Computed Class Weights: {0: np.float64(1.9448173005219984), 1: np.float64(0.6730322580645162)}


In [15]:
# AUTOENCODER DEFINITION (simple conv autoencoder)
# - This autoencoder will learn to reconstruct the input image.
# - After training we'll use autoencoder.predict(X) to obtain denoised/reconstructed images.
# =========================
ae_input = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Encoder: reduce spatial size and increase channels
x = Conv2D(32, (3, 3), activation='relu', padding='same')(ae_input)  # out: 224x224x32
x = MaxPooling2D((2, 2), padding='same')(x)                           # out: 112x112x32
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)         # out: 112x112x64
encoded = MaxPooling2D((2, 2), padding='same')(x)                    # out: 56x56x64  <-- latent

# Decoder: reconstruct back to original size
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)   # out: 56x56x64
x = UpSampling2D((2, 2))(x)                                          # out: 112x112x64
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)         # out: 112x112x32
x = UpSampling2D((2, 2))(x)                                          # out: 224x224x32
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x) # out: 224x224x3

autoencoder = Model(ae_input, decoded, name='conv_autoencoder')
autoencoder.compile(optimizer='adam', loss='mse')  # MSE between input and reconstruction
print("\n===== Autoencoder Summary =====")
autoencoder.summary()



===== Autoencoder Summary =====


Model: "conv_autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 56, 56, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 112, 112, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 224, 224, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75,651 (295.51 KB)

 Trainable params: 75,651 (295.51 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# AUTOENCODER TRAINING DATA
# =========================
ae_train_gen_raw = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode=None, shuffle=True
)
ae_val_gen_raw = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode=None, shuffle=True
)

def autoencoder_generator(gen):
    for batch in gen:
        yield (batch, batch)

print("\nTraining Autoencoder...")
autoencoder.fit(
    autoencoder_generator(ae_train_gen_raw),
    steps_per_epoch=len(ae_train_gen_raw),
    epochs=5,
    validation_data=autoencoder_generator(ae_val_gen_raw),
    validation_steps=len(ae_val_gen_raw)
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.

Training Autoencoder...
Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 347s 2s/step - loss: 0.0086 - val_loss: 0.0018
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 417s 3s/step - loss: 0.0015 - val_loss: 0.0014
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 439s 3s/step - loss: 0.0012 - val_loss: 0.0012
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 585s 4s/step - loss: 0.0010 - val_loss: 0.0011
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 563s 3s/step - loss: 9.4951e-04 - val_loss: 9.3152e-04


In [9]:
# TRAIN AUTOENCODER
# - We train only briefly here (5 epochs) — increase for production.
# - We use the wrapped generator so the model sees (inputs, targets) both images.
# =========================
print("\nTraining Autoencoder...")
autoencoder.fit(
    autoencoder_generator(ae_train_gen_raw),
    steps_per_epoch=ae_steps_per_epoch,
    epochs=5,  # increase to 10-50 for stronger denoising/feature learning
    validation_data=autoencoder_generator(ae_val_gen_raw),
    validation_steps=ae_val_steps
)


Training Autoencoder...
Epoch 1/5
  2/163 ━━━━━━━━━━━━━━━━━━━━ 4:52 2s/step - loss: 0.0549

KeyboardInterrupt: 

In [ ]:
# After AE training, we will reconstruct images (autoencoder.predict) for train/val/test
# =========================
# FUNCTION: build reconstructed arrays from a labeled generator
# - Iterates one epoch through the labeled generator (train_generator / validation_generator / test_generator)
# - Runs autoencoder.predict on each image batch to get reconstructed images
# - Returns numpy arrays: X_recon, y_labels
# NOTE: This will load reconstructed images into memory. If dataset is huge, convert to
#       on-disk TFRecords or use streaming pipelines instead.
# =========================
def build_reconstructed_dataset(labeled_gen, model_ae, batch_limit=None):
    """Return (X_recon, y) by running model_ae.predict on each batch from labeled_gen.
       If batch_limit is given, use that many batches (useful for debugging)."""
    x_list = []
    y_list = []
    # When using flow_from_directory, one full iteration uses len(generator) steps.
    steps = batch_limit if batch_limit is not None else len(labeled_gen)
    # Reset generator index
    labeled_gen.reset()
    for i in range(steps):
        x_batch, y_batch = next(labeled_gen)  # x_batch: (B, 224,224,3), y_batch: (B,)
        # predict reconstructed images for this batch (autoencoder outputs in range [0,1])
        x_recon = model_ae.predict(x_batch, verbose=0)
        x_list.append(x_recon)
        y_list.append(y_batch)
    X_recon = np.concatenate(x_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    return X_recon, y

print("\nReconstructing train/val/test images using trained autoencoder (this may take a while)...")
# Build reconstructed versions for train, val, test (one pass through each generator)
X_train_recon, y_train = build_reconstructed_dataset(train_generator, autoencoder)
X_val_recon,   y_val   = build_reconstructed_dataset(validation_generator, autoencoder)
X_test_recon,  y_test  = build_reconstructed_dataset(test_generator, autoencoder)

print("Shapes - X_train_recon:", X_train_recon.shape, "y_train:", y_train.shape)
print("Shapes - X_val_recon:", X_val_recon.shape, "y_val:", y_val.shape)
print("Shapes - X_test_recon:", X_test_recon.shape, "y_test:", y_test.shape)


Reconstructing train/val/test images using trained autoencoder (this may take a while)...
Shapes - X_train_recon: (5216, 224, 224, 3) y_train: (5216,)
Shapes - X_val_recon: (16, 224, 224, 3) y_val: (16,)
Shapes - X_test_recon: (624, 224, 224, 3) y_test: (624,)


In [ ]:
# RESNET50 (classifier) - now receives reconstructed images as input
# =========================
print("\nBuilding ResNet50 classifier (input = reconstructed images)...")
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  # freeze base initially

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)
predictions = Dense(1, activation='sigmoid')(x)

classifier = Model(inputs=base_model.input, outputs=predictions, name='resnet_classifier')
classifier.compile(optimizer=Adam(learning_rate=1e-3),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])
print("\n===== Classification Model Summary =====")
classifier.summary()



Building ResNet50 classifier (input = reconstructed images)...

===== Classification Model Summary =====


Model: "resnet_classifier"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,589,761 (89.99 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# CALLBACKS
# =========================
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=3,
    verbose=1,
    min_lr=1e-7
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)


In [ ]:
# INITIAL TRAINING (on reconstructed images)
# - Here we pass numpy arrays directly. If memory is an issue, build a tf.data pipeline
#   or reconstruct batches on the fly.
# =========================
print("\nTraining classifier on reconstructed images...")
history = classifier.fit(
    X_train_recon, y_train,
    batch_size=BATCH_SIZE,
    epochs=10,
    validation_data=(X_val_recon, y_val),
    class_weight=class_weights,
    callbacks=[lr_scheduler, early_stop]
)


Training classifier on reconstructed images...
Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 385s 2s/step - accuracy: 0.5194 - loss: 0.7043 - val_accuracy: 0.5000 - val_loss: 0.6905 - learning_rate: 0.0010
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 358s 2s/step - accuracy: 0.5709 - loss: 0.6764 - val_accuracy: 0.7500 - val_loss: 0.6646 - learning_rate: 0.0010
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 329s 2s/step - accuracy: 0.5957 - loss: 0.6602 - val_accuracy: 0.5000 - val_loss: 0.7003 - learning_rate: 0.0010
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 369s 2s/step - accuracy: 0.6735 - loss: 0.6386 - val_accuracy: 0.5625 - val_loss: 0.6518 - learning_rate: 0.0010
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 387s 2s/step - accuracy: 0.6720 - loss: 0.6266 - val_accuracy: 0.6250 - val_loss: 0.6385 - learning_rate: 0.0010
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 388s 2s/step - accuracy: 0.6865 - loss: 0.6221 - val_accuracy: 0.6250 - val_loss: 0.6344 - learning_rate: 0.0010
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━

In [ ]:
# FINE-TUNING
# - Unfreeze last 30 layers of ResNet50 and continue training with small LR
# =========================
print("\nFine-tuning classifier (unfreezing last 30 layers)...")
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

classifier.compile(optimizer=Adam(learning_rate=1e-5),
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

fine_tune_history = classifier.fit(
    X_train_recon, y_train,
    batch_size=BATCH_SIZE,
    epochs=5,
    validation_data=(X_val_recon, y_val),
    class_weight=class_weights,
    callbacks=[lr_scheduler, early_stop]
)



Fine-tuning classifier (unfreezing last 30 layers)...
Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 466s 3s/step - accuracy: 0.8092 - loss: 0.6248 - val_accuracy: 0.5000 - val_loss: 2.3102 - learning_rate: 1.0000e-05
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 405s 2s/step - accuracy: 0.8944 - loss: 0.2598 - val_accuracy: 0.6250 - val_loss: 0.6574 - learning_rate: 1.0000e-05
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8895 - loss: 0.2526
Epoch 3: ReduceLROnPlateau reducing learning rate to 2.9999999242136253e-06.
163/163 ━━━━━━━━━━━━━━━━━━━━ 404s 2s/step - accuracy: 0.8976 - loss: 0.2316 - val_accuracy: 0.6875 - val_loss: 0.7161 - learning_rate: 1.0000e-05
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 408s 3s/step - accuracy: 0.9308 - loss: 0.1784 - val_accuracy: 0.6250 - val_loss: 0.5786 - learning_rate: 3.0000e-06
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 414s 3s/step - accuracy: 0.9390 - loss: 0.1562 - val_accuracy: 0.8125 - val_loss: 0.7047 - learning_rate: 3.0000e-06
Restoring mode

In [ ]:
# EVALUATE ON TEST DATA
# =========================
test_loss, test_acc = classifier.evaluate(X_test_recon, y_test, batch_size=BATCH_SIZE)
print(f"\n✅ Test Accuracy on reconstructed test set: {test_acc:.4f}")

NameError: name 'classifier' is not defined